In [1]:
import sys
sys.path.append("../")
import nltk
import pandas as pd
import numpy as np
from helper import tokenize, dicttodf
import json
from nltk.tokenize import RegexpTokenizer
import shutil
import os

In [2]:
path = "/Users/sebastiaanscholten/Documents/speech2image-master/vgsexperiments/experiments/data/Flickr8k_text/Flickr8k.token.txt"

In [3]:
path2 = "/Users/sebastiaanscholten/Documents/speech2image-master/vgsexperiments/experiments/data/dataset.json"
file = json.load(open(path2))
split_dict = {}
for x in file['images']:
    split_dict[x['filename'].replace('.jpg', '')] = x['split']

In [4]:
testset = []
for key, value in split_dict.items():
    if value == "test":
        testset.append(key)

In [5]:
text = pd.read_csv(path, sep="\t",header=None)
text.columns = ["images","captions"]
textcopy = text
text.images = [words[:-6] for words in text.images if words==words]
text = text[text.images.isin(testset)]
text

,images,captions
210,1056338697_4f7d7ce270,A blond woman in a blue shirt appears to wait ...
211,1056338697_4f7d7ce270,A blond woman is on the street hailing a taxi .
212,1056338697_4f7d7ce270,"A woman is signaling is to traffic , as seen f..."
213,1056338697_4f7d7ce270,A woman with blonde hair wearing a blue tube t...
214,1056338697_4f7d7ce270,The woman in the blue dress is holding out her...
...,...,...
40455,997722733_0cb5439472,A man in a pink shirt climbs a rock face
40456,997722733_0cb5439472,A man is rock climbing high in the air .
40457,997722733_0cb5439472,A person in a red shirt climbing up a rock fac...
40458,997722733_0cb5439472,A rock climber in a red shirt .


In [6]:
text = text["captions"]
tokenizer = RegexpTokenizer(r'\w+')
wordlist = []
for sentence in text:
    sentence = sentence.lower()
    words = tokenizer.tokenize(sentence)
    wordlist.extend(words)
wordlist = [x.lower() for x in wordlist]
sr = pd.Series(wordlist)
worddictionary = dict(sr.value_counts())



In [7]:
worddictionary

{'a': 7823,
 'the': 2383,
 'in': 2319,
 'on': 1386,
 'is': 1183,
 'and': 1044,
 'with': 973,
 'dog': 953,
 'man': 933,
 'of': 811,
 'two': 696,
 'white': 528,
 'boy': 507,
 'black': 479,
 'are': 456,
 'girl': 427,
 'woman': 388,
 'wearing': 388,
 'to': 383,
 'at': 374,
 'water': 352,
 'people': 341,
 'red': 339,
 'young': 328,
 'his': 325,
 'blue': 303,
 'an': 295,
 'playing': 278,
 'while': 264,
 'dogs': 263,
 'brown': 256,
 'little': 255,
 'shirt': 240,
 'standing': 238,
 'through': 237,
 'down': 226,
 'running': 220,
 'child': 214,
 'ball': 209,
 'person': 198,
 'front': 192,
 'grass': 188,
 'over': 186,
 'small': 185,
 'jumping': 181,
 'beach': 174,
 'up': 172,
 'holding': 169,
 'sitting': 169,
 'three': 163,
 'snow': 160,
 'her': 159,
 'by': 159,
 'green': 153,
 'one': 149,
 'walking': 149,
 'into': 148,
 'large': 148,
 'air': 145,
 'group': 142,
 'near': 140,
 'yellow': 139,
 'children': 137,
 'field': 135,
 'men': 131,
 'street': 122,
 'jumps': 121,
 'another': 119,
 'as': 118,


In [8]:
# import nltk
# postags = nltk.pos_tag(worddf["words"])

In [9]:
from helper import return_postag_keys


In [10]:
textcp = pd.read_csv(path, sep="\t",header=None)
textcp.columns = ["images","captions"]
textcp["original"] = textcp["images"]
textcp.images = [words[:-6] for words in textcp.images if words==words]
textcp = textcp[textcp.images.isin(testset)]
textcp = textcp.drop(columns = "images")
textcp
textcp.to_csv("/Users/sebastiaanscholten/Documents/speech2image-master/vgsexperiments/experiments/Results_isolated_word_recognition/documents/textcp.csv",index=False)

In [11]:
def match_word_wavfile():
    inputword = input("Enter word here: ")
    wavfiles = []
    for idx, captions in enumerate(textcp.captions):
        if inputword in tokenizer.tokenize(captions):
            wavfiles.append(textcp.original.iloc[idx])
    wavfilenames = []
    for word in wavfiles:
        word = word.replace("#","_")
        word = word.replace(".jpg","")
        wavfilenames.append(word)
    return wavfilenames,inputword,wavfiles
        
    

In [12]:
wavfilenames,inputword,wavfiles = match_word_wavfile()


Enter word here: 


In [13]:
len(wavfilenames)

0

In [14]:
# def seperate_files_from_list(inputwords):
#     #loop through the words
#     for inputword in inputwords:
#         wavfiles = []
#         #find captions that contain these words
#         for idx, captions in enumerate(textcp.captions):
#             if inputword in tokenizer.tokenize(captions):
#                 wavfiles.append(textcp.original.iloc[idx])
#         wavfilenames = []
#         #preprocess them for file copying
#         for word in wavfiles:
#             word = word.replace("#","_")
#             word = word.replace(".jpg","")
#             wavfilenames.append(word)
#         count = 0
#         source = "/Volumes/Schnellheit/ThesisData/flickrwavsandimagestextandjson/flickr_audio/wavs/"
#         dest = "/Volumes/Schnellheit/ThesisData/flickrwavsandimagestextandjson/flickr_audio/" + inputword + "/"
#         #move the files
#         if not os.path.exists(dest):
#             os.mkdir(dest)
#         for word in wavfilenames:
#             count = count + 1
#             word = word + ".wav"
#             sourceword = source + word
#             worddest = dest + word
#             shutil.copy(sourceword,worddest)
#             if count == 50:
#                 count = 0
#                 break
# listtest = ["dog","man","boy","girl","woman","people"
#             ,"dogs","shirt","child","ball","person"
#             ,"children","men","girls","bike","rock","camera"
#             ,"boys","hat","player","jacket","basketball","swing"
#             ,"car", "wall", "hair","football","sunglasses","head"
#             ,"shorts","dress","table"]

# # seperate_files_from_list(listtest)

In [15]:
from collections import defaultdict
def print_file_list(inputwords):
    #loop through the words
    word_appended_finallist = []
    finallist = []
    finallistdict = defaultdict(list)
    for inputword in inputwords:
        wavfiles = []
        #find captions that contain these words
        for idx, captions in enumerate(textcp.captions):
            if inputword in tokenizer.tokenize(captions):
                wavfiles.append(textcp.original.iloc[idx])
        wavfilenames = []
        #preprocess them for file copying
        for word in wavfiles:
            word = word.replace("#","_")
            word = word.replace(".jpg","")
            wordwithinputword = inputword + "_" + word
            wavfilenames.append(word)
        count = 0
        #move the files
        for word in wavfilenames:
            count = count + 1
            word_appended_finallist.append(wordwithinputword)
            finallist.append(word)
            file = word.split('_', -1)[0:2]
            testword = "_".join(file)
            wordjpg = testword + ".jpg"
            finallistdict[wordjpg].append(inputword)
            if count == 50:
                count = 0
                break         
    return wavfiles,wavfilenames,word_appended_finallist,finallist,finallistdict

In [16]:
listtest2 = ["dog","man","boy","girl","woman","people"
            ,"dogs","shirt","child","ball","person"
            ,"children","men","girls","bike","rock","camera"
            ,"boys","hat","player","jacket","basketball","swing"
            ,"car", "wall", "hair","football","sunglasses","head"
            ,"shorts","dress","table","water","grass","bench","snow"
            ,"air","field","street","mouth","dirt","mountain","pool"
            ,"ocean","sand","building","soccer","park","face"]

In [17]:
len(listtest2)

49

In [18]:
wavfiles, wavfilenames,word_appended_finallist,finallist,finallistdict = print_file_list(listtest2)


In [19]:
word_appended_finallist = pd.DataFrame(word_appended_finallist)
word_appended_finallist.to_csv("word_appended_finallist.txt",index=False,header=False)

finallist = pd.DataFrame(finallist)
finallist.to_csv("finallist.txt",index=False,header=False)

In [20]:
finallistdict


defaultdict(list,
            {'1084040636_97d9633581.jpg': ['dog',
              'dog',
              'dog',
              'dog',
              'dog',
              'head',
              'head',
              'head',
              'head'],
             '1107246521_d16a476380.jpg': ['dog',
              'dog',
              'dog',
              'dog',
              'dog',
              'ball'],
             '1119015538_e8e796281e.jpg': ['dog',
              'dog',
              'dog',
              'dog',
              'dogs',
              'grass',
              'grass',
              'grass',
              'field'],
             '1131800850_89c7ffd477.jpg': ['dog',
              'dog',
              'dog',
              'dog',
              'dog',
              'water',
              'water',
              'water'],
             '1267711451_e2a754b4f8.jpg': ['dog',
              'dog',
              'dog',
              'dog',
              'dog',
              'grass',
             

In [ ]:
# #inefficient way to remove caption number from files for comparison
# file = captionfile.split('_', -1)[0:2]
# testword = "_".join(file)


In [ ]:
with open('/Users/sebastiaanscholten/Documents/speech2image-master/vgsexperiments/experiments/data/finallistdict.json', 'w') as fp:
    json.dump(finallistdict, fp)
    

In [21]:
len(finallistdict)

580